In [1]:
#Load packages
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [3]:
# Scrape a league matches from understat
base_url = 'https://understat.com/league/'
# Select the English Premier League
league = 'EPL' 
url = base_url+league

In [5]:
#Use requests to get the webpage and BeautifulSoup to parse the page
res = requests.get(url)
soup = BeautifulSoup(res.content)
scripts = soup.find_all('script')

In [7]:
#Get only the dates Data
strings = scripts[1].string

In [9]:
# Clean from unnecessary symbols and get only JSON data 
str_start = strings.index("('")+2 
str_end = strings.index("')") 
json_data = strings[str_start:str_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

# Convert JSON string
EPL_data = json.loads(json_data)
EPL_data[0]

{'id': '26602',
 'isResult': True,
 'h': {'id': '89', 'title': 'Manchester United', 'short_title': 'MUN'},
 'a': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'},
 'goals': {'h': '1', 'a': '0'},
 'xG': {'h': '2.04268', 'a': '0.418711'},
 'datetime': '2024-08-16 19:00:00',
 'forecast': {'w': '0.8069', 'd': '0.1489', 'l': '0.0442'}}

In [11]:
# Separate home and away teams into different DataFrames
df = pd.json_normalize(EPL_data, sep="_")
df.head()

,id,isResult,datetime,h_id,h_title,h_short_title,a_id,a_title,a_short_title,goals_h,goals_a,xG_h,xG_a,forecast_w,forecast_d,forecast_l
0,26602,True,2024-08-16 19:00:00,89,Manchester United,MUN,228,Fulham,FLH,1,0,2.04268,0.418711,0.8069,0.1489,0.0442
1,26603,True,2024-08-17 11:30:00,285,Ipswich,IPS,87,Liverpool,LIV,0,2,0.342601,3.92906,0.0026,0.018,0.9794
2,26604,True,2024-08-17 14:00:00,83,Arsenal,ARS,229,Wolverhampton Wanderers,WOL,2,0,1.6283,0.575835,0.6528,0.231,0.1162
3,26605,True,2024-08-17 14:00:00,72,Everton,EVE,220,Brighton,BRI,0,3,0.405325,1.79083,0.0481,0.1635,0.7884
4,26606,True,2024-08-17 14:00:00,86,Newcastle United,NEW,74,Southampton,SOU,1,0,0.433489,1.95483,0.0378,0.164,0.7982


In [13]:
# Check types
df.dtypes

id               object
isResult           bool
datetime         object
h_id             object
h_title          object
h_short_title    object
a_id             object
a_title          object
a_short_title    object
goals_h          object
goals_a          object
xG_h             object
xG_a             object
forecast_w       object
forecast_d       object
forecast_l       object
dtype: object

In [15]:
# Data cleansing and converting data types
df.fillna(0, inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.normalize()
df[['id', 'h_id', 'a_id', 'goals_h', 'goals_a']] = df[['id', 'h_id', 'a_id', 'goals_h', 'goals_a']].astype(int)
df[['xG_h', 'xG_a', 'forecast_w', 'forecast_d', 'forecast_l']] = df[['xG_h', 'xG_a', 'forecast_w', 'forecast_d', 'forecast_l']].astype(float)

In [17]:
df.head()

,id,isResult,datetime,h_id,h_title,h_short_title,a_id,a_title,a_short_title,goals_h,goals_a,xG_h,xG_a,forecast_w,forecast_d,forecast_l,date
0,26602,True,2024-08-16 19:00:00,89,Manchester United,MUN,228,Fulham,FLH,1,0,2.042680,0.418711,0.8069,0.1489,0.0442,2024-08-16
1,26603,True,2024-08-17 11:30:00,285,Ipswich,IPS,87,Liverpool,LIV,0,2,0.342601,3.929060,0.0026,0.0180,0.9794,2024-08-17
2,26604,True,2024-08-17 14:00:00,83,Arsenal,ARS,229,Wolverhampton Wanderers,WOL,2,0,1.628300,0.575835,0.6528,0.2310,0.1162,2024-08-17
3,26605,True,2024-08-17 14:00:00,72,Everton,EVE,220,Brighton,BRI,0,3,0.405325,1.790830,0.0481,0.1635,0.7884,2024-08-17
4,26606,True,2024-08-17 14:00:00,86,Newcastle United,NEW,74,Southampton,SOU,1,0,0.433489,1.954830,0.0378,0.1640,0.7982,2024-08-17
